In [15]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable
import json

class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    def create_friendship(self, person1_name, person2_name):
        with self.driver.session(database="neo4j") as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.write_transaction(
                self._create_and_return_friendship, person1_name, person2_name)
            for row in result:
                print("Created friendship between: {p1}, {p2}".format(p1=row['p1'], p2=row['p2']))

    def remove_all(self):
        with self.driver.session(database="neo4j") as session:
            session.run("MATCH (n) DETACH DELETE n")
            print("All nodes and edges are removed")

        # set the constraint for id
        with self.driver.session(database="neo4j") as session:
            session.run("CREATE CONSTRAINT unique_profile_id IF NOT EXISTS FOR (p:Person) REQUIRE p.id IS UNIQUE")
            print("Constraint is set")


    def number_of_nodes_edges(self):
        with self.driver.session(database="neo4j") as session:
            # find number of nodes and edges
            result_node = session.run("MATCH (n) RETURN count(n) as nodes")
            result_edge = session.run("MATCH ()-[r]->() RETURN count(r) as edges")
            for row in result_node:
                print("Number of nodes: {nodes}".format(nodes=row['nodes']))
            for row in result_edge:
                print("Number of edges: {edges}".format(edges=row['edges']))
            
            
        

    @staticmethod
    def _create_and_return_friendship(tx, person1_id, person2_id):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        query = (
            "MERGE (p1:Person { id: $person1_id}) "
            "MERGE (p2:Person { id: $person2_id}) "
            "MERGE (p1)-[:KNOWS]->(p2) "
            "RETURN p1, p2"
        )
        result = tx.run(query, person1_id=person1_id, person2_id=person2_id)
        try:
            return [{"p1": row["p1"]["id"], "p2": row["p2"]["id"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    @staticmethod
    def _find_and_return_person(tx, person_id):
        query = (
            "MATCH (p:Person) "
            "WHERE p.id = $person_id "
            "RETURN p.id AS id"
        )
        result = tx.run(query, person_id=person_id)
        return [row["id"] for row in result]


if __name__ == "__main__":
    # Aura queries use an encrypted connection using the "neo4j+s" URI scheme
    uri = "Change it with the URI of your Aura database"
    user = "neo4j"
    password = "Change it with the password of your Aura database"
    app = App(uri, user, password)
    app.remove_all()
    print("Number of nodes and edges after removing all:")
    app.number_of_nodes_edges()
    # open Sample.json and load data
    with open('Sample.json') as f:
        data = json.load(f)
    # create friendship between person1 and person2 from edges
    for i in data['edges']:
        app.create_friendship(i['data']["source"], i['data']["target"])
    
    print("Number of nodes and edges after loading data:")
    app.number_of_nodes_edges()
    app.close()



All nodes and edges are removed
Constraint is set
Number of nodes and edges after removing all:
Number of nodes: 0
Number of edges: 0
Created friendship between: n4, n5
Created friendship between: n4, n6
Created friendship between: n6, n7
Created friendship between: n6, n8
Created friendship between: n5, n7
Number of nodes and edges after loading data:
Number of nodes: 5
Number of edges: 5
